# AD Bracketing

First, initialise the needed libraries.

In [133]:
{-# LANGUAGE OverloadedStrings, TypeApplications #-}

import Control.Lens
import Data.Foldable
import Data.Semigroup
import Control.Applicative
import Math.AD.Data.Bracketing
import Math.AD.Real.FuncDescription
import Math.AD.Real.FuncChain
import Data.Traversable.Enumerate
import Text.LaTeX (Texy, texy, LaTeX, (!:), mathsf, math)

F (15,26) 154

In [23]:
edgesMatProd :: FuncDescription a => a -> Integer
edgesMatProd x = inDim x * outDim x

edgesMatProd (F (15,16) 124)

fDescFromBase :: FuncDescription a => a -> Integer -> FDesc'
fDescFromBase x e = F (inDim x, outDim x) (e * outDim x + edgesMatProd x)

fe = 5
ge = 3
he = 9
ie = 7
je = 11
ke = 1
le = 9

baseEdges = [fe, ge, he, ie, je, ke, le]

zipWith fDescFromBase (replicate (length baseEdges) (F (16,16) 0)) baseEdges
 

240

In [137]:
propDims = DimList [16,8,4,1] :: DimList'

inputs = [ke, ke, je, le]

fList = zipWith fDescFromBase (init $ descList propDims) (init $ tail inputs)

initFDesc = F(16,16) (head inputs * 16) :: FDesc'
lDesc = F (4,1) (last inputs) :: FDesc'

caseDescList = initFDesc:(fList ++ [lDesc])

toFChain caseDescList

# Functions on bracketings

In [69]:
fmaMatProd :: FuncDescription a => a -> a -> Integer
fmaMatProd lhs rhs = inDim rhs * inDim lhs * outDim lhs

fma :: FuncDescription a => Bracketing a -> Integer
fma Start = 0
fma (Tangent f Start) = inDim f  * edgeCount f
fma (Adjoint Start f) = outDim f * edgeCount f
fma (Preacc Start rhs) = fma rhs
fma (Preacc lhs Start) = fma lhs
fma (Tangent f other) = fma other + inDim other  * edgeCount f
fma (Adjoint other f) = fma other + outDim other * edgeCount f
fma (Preacc  lhs rhs) = fma lhs + fma rhs + fmaMatProd lhs rhs

mem :: FuncDescription a => Bracketing a -> Integer
mem Start = 0
mem (Tangent f other) = max (mem other) (inDim f)
mem (Adjoint other f) = mem other + edgeCount f
mem (Preacc  lhs rhs) = max (mem lhs) (mem rhs) + outDim lhs * inDim rhs

## Generating all possible bracketings.


In [138]:
bracketings = genEach caseDescList

withPlaceholders :: Bracketing a -> Bracketing LaTeX
withPlaceholders x = enumMap f x 1 where
    f :: a -> Integer -> LaTeX
    f _ e = "F" !: fromInteger e

length bracketings

argMinimum :: (Monoid a, Ord b) => (a -> b) -> [a] -> a
argMinimum f xs = fold $ minimum $ (\x -> Arg (f x) x) <$> xs

argMaximum :: (Monoid a, Ord b) => (a -> b) -> [a] -> a
argMaximum f xs = fold $ maximum $ (\x -> Arg (f x) x) <$> xs

brakSpaces = (\f -> f bracketings) <$> fs where
    fs = ((\u -> filter (\x -> mem x < u)) <$> [50,60..500]) ++ [id]

texyResult :: (FuncDescription a) => [Bracketing a] -> LaTeX
texyResult xs = mathsf "best:" <> texy (withPlaceholders best) <> mathsf ", fma=" <> texy (fma best) <> mathsf ", mem=" <> texy (mem best) where
    best = argMinimum fma xs
    
math.texyResult <$> brakSpaces

352

76

[(1440,160),(313,313)]